# ✅ **Day 6: Building a Data Pipeline — Full Breakdown**

We’ll build a **Mini ML Pipeline** that covers **every step from raw data to deployable prediction output**.

---

## 📌 **What is a Data Pipeline?**

A **data pipeline** is a *series of steps that process raw data* into usable, clean, and model-ready formats — and finally turns them into outputs like predictions, files, dashboards, or databases.

It brings **structure, repeatability, and automation** to your ML workflow.

Think of it like a **conveyor belt**:

> 🚛 Ingest → 🧽 Clean → 🤖 Model → 💾 Save → 📤 Export

---

### 🧱 **Steps in Our Pipeline**

Let’s break down each step we’ll be implementing today:

---

### **🔹 1. Ingest Data (Load)**
    
---

### **🔹 2. Clean & Preprocess Data**

---

### **🔹 3. Model Training (XGBoost)**

---

### **🔹 4. Save Model (joblib)**

---

### **🔹 5. Export Output (CSV/Excel/DB)**

---

### 🛠️ **Tools You’ll Use**

| Tool        | Purpose                   |
| ----------- | ------------------------- |
| `pandas`    | Data loading & cleaning   |
| `xgboost`   | ML modeling               |
| `joblib`    | Save/load models          |
| `sklearn`   | Train-test split, metrics |
| `CSV/Excel` | Save outputs              |

---

### ✅ **Today’s Final Deliverable:**

A **complete pipeline notebook** that:

1. Loads raw data

2. Preprocesses it

3. Trains an XGBoost model

4. Saves the trained model

5. Saves predictions as a file

---

## ✅ Step 1: Ingest Data

Let’s use the **Titanic Dataset** — a very popular and clean dataset used in many real-world ML examples.

It’s perfect for practicing pipelines because it has:

* Both **numerical** and **categorical** features

* **Missing values**

* A **classification target** (`Survived`: 0 or 1)

---

### 📂 Dataset Overview:

| Column        | Description                     |
| ------------- | ------------------------------- |
| `PassengerId` | Passenger number (irrelevant)   |
| `Pclass`      | Ticket class (1, 2, 3)          |
| `Name`        | Name of the passenger           |
| `Sex`         | Gender                          |
| `Age`         | Age (some missing values)       |
| `SibSp`       | # of siblings / spouses aboard  |
| `Parch`       | # of parents / children aboard  |
| `Ticket`      | Ticket number                   |
| `Fare`        | Passenger fare                  |
| `Cabin`       | Cabin number (many missing)     |
| `Embarked`    | Port of Embarkation             |
| `Survived`    | **Target**: 0 = Died, 1 = Lived |

---


In [2]:
import pandas as pd

# Load dataset from GitHub (no need to download manually)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# View first few rows
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## ✅ Step 2: Clean & Preprocess the Titanic Data

### 1️⃣ **Drop Unnecessary Columns**

* `PassengerId`, `Name`, `Ticket`, and `Cabin` are not useful for prediction.
* `Cabin` also has too many missing values.

In [6]:
df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)

### 2️⃣ **Handle Missing Values**

In [8]:
#Let’s check how many are missing first:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

* `Age` – fill with median
* `Embarked` – fill with mode

In [11]:
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

C:\Users\lucky\AppData\Local\Temp\ipykernel_18604\1139536044.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)
C:\Users\lucky\AppData\Local\Temp\ipykernel_18604\1139536044.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

### 3️⃣ **Encode Categorical Variables**

This will convert:

* `Sex` → `Sex_male`

* `Embarked` → `Embarked_Q`, `Embarked_S`

* Drops the first category to avoid multicollinearity.

In [12]:
df = pd.get_dummies(df, drop_first=True)

# When we use pd.get_dummies() to convert categorical variables into numeric columns, it creates one column for each category.
# drop_first=True means Drop the first category to avoid multicollinearity (duplicate information).

### ✅ Final Cleaned Data Check

In [16]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,True,False,True
1,1,1,38.0,1,0,71.2833,False,False,False
2,1,3,26.0,0,0,7.9250,False,False,True
3,1,1,35.0,1,0,53.1000,False,False,True
4,0,3,35.0,0,0,8.0500,True,False,True


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Sex_male    891 non-null    bool   
 7   Embarked_Q  891 non-null    bool   
 8   Embarked_S  891 non-null    bool   
dtypes: bool(3), float64(2), int64(4)
memory usage: 44.5 KB


## 🔹Step 3. Model Training (XGBoost)

In [22]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Features and Target
x = df.drop("Survived", axis=1)
y = df["Survived"]

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train XGBoost Classifier
model = XGBClassifier()
model.fit(x_train, y_train)

# Test and Evaluate
y_pred = model.predict(x_test)

# Accuracy
print("Test Acuuracy :", accuracy_score(y_test, y_pred))

# Confusion Matrix
print("Confusion Metrics :", confusion_matrix(y_test, y_pred))

Test Acuuracy : 0.8044692737430168
Confusion Metrics : [[88 17]
 [18 56]]


### ✅ **Model Accuracy**

Test Accuracy : 0.8044692737430168

That means your XGBoost classifier correctly predicted the survival of passengers **\~80.4%** of the time on unseen test data.

For a real-world Titanic dataset with limited features and size, this is **pretty good**!

---

### 📊 **Confusion Matrix**

A confusion matrix shows us how well the model is performing in terms of **true/false predictions**:

|                          | **Predicted: Died (0)** | **Predicted: Survived (1)** |
| ------------------------ | ----------------------- | --------------------------- |
| **Actual: Died (0)**     | 88 ✅ (True Negative)    | 17 ❌ (False Positive)       |
| **Actual: Survived (1)** | 18 ❌ (False Negative)   | 56 ✅ (True Positive)        |

### 🧠 Interpretation:

* ✅ **88 people** who died were correctly predicted as died.

* ✅ **56 people** who survived were correctly predicted as survived.

* ❌ **17 people** were wrongly predicted to survive, but they actually died.

* ❌ **18 people** were wrongly predicted to die, but they actually survived.

---

## 🔹 Step 4: Save Model with `joblib`

---

### ✅ **What is `joblib`? Why do we use it?**

* `joblib` is a library for **efficiently saving and loading Python objects**, especially:

  * Machine learning models
  * Large NumPy arrays
  * Pipelines

⚡ It’s faster and more reliable than `pickle` for large data.


In [24]:
# Step 1: Import the joblib library
import joblib #Loads the joblib module so we can use its dump() function.

# Step 2: Save the trained model as a .pkl file (Pickle format)
joblib.dump(model, "churn_model.pkl") #Saves your trained XGBClassifier model to a file named 'churn_model.pkl'.

#After saving, you can reuse the model later (without retraining) like this:
# Load the saved model
#loaded_model = joblib.load("churn_model.pkl")

# Use it for prediction
#loaded_model.predict(x_test)

['churn_model.pkl']

## 🔹 **Step 5: Export Output (CSV / Excel / SQLite)**

### ✅ Goal:

You’ve trained your model, evaluated it, and now you’ll **save the predictions** to a file you can share, analyze further, or use in reports.

In [25]:
# Step 1: Create a new DataFrame with actual and predicted labels
output_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})

# Step 2: Save the predictions to a CSV file
output_df.to_csv("Predictions.csv", index=False)

#Optional:

#Want to save to Excel?
output_df.to_excel("Predictions.xlsx", index=False)

#Want to save to SQLite?
import sqlite3 #This line imports Python’s built-in sqlite3 module, which lets you create and interact with SQLite databases (lightweight, file-based databases).

conn = sqlite3.connect("ml_outputs.db") #Creates a connection object called conn to a new or existing database file named ml_outputs.db.

output_df.to_sql("titanic_predictions", conn, index=False, if_exists="replace")
# 'titanic_predictions'	The name of the table in the database
# conn	The connection to the database
# index=False	Do not save the Pandas index as a column
# if_exists='replace'	If a table with this name already exists, replace it

conn.close()